In [ ]:
import sys, os
sys.path.append(os.path.join(os.path.expanduser('~'), 'work', 'physion', 'src'))
import physion
import physion.utils.plot_tools as pt
import pandas as pd
import numpy as np

In [ ]:
datafolder = os.path.join(os.path.join(os.path.expanduser('~'), 'DATA', 'Cibele', 'PV_BB_V1', 'NWBs'))
protocol = 'ff-gratings-8orientation-2contrasts-15repeats'
DATASET = physion.analysis.read_NWB.scan_folder_for_NWBfiles(datafolder,
                                                  for_protocol=protocol, sorted_by='age')
df = pd.DataFrame({'file':[os.path.basename(f) for f in DATASET['files']],
                  'subject':DATASET['subjects'],
                  'protocol':[p[0] for p in DATASET['protocols']],
                  'age':DATASET['ages']})
df

In [ ]:
fig, ax = pt.figure(figsize=(3,3))

ax.axis('off')
(0.0, 1.0, 0.0, 1.0)

table = pd.plotting.table(ax, df) #, loc='center',                           cellLoc='center', colWidths=list([.2, .2]))


In [ ]:
from physion.analysis.protocols.orientation_tuning import compute_tuning_response_per_cells


stat_test = dict(interval_pre=[-1.5,-0.5],
                 interval_post=[0.5,1.5],
                 test='ttest',
                 positive=True)
response_significance_threshold=0.05

for contrast in [0.5, 1.0]:
    
    for key in ['fraction_responsive', 'responsive',
                'mean-selectivity', 'sd-selectivity',
                'tuning-curves']:
        DATASET[key+'-c=%.1f'%contrast] = []
    
    for f in DATASET['files']:
        
        data = physion.analysis.read_NWB.Data(f, verbose=False)
        Episodes = physion.analysis.process_NWB.EpisodeData(data, quantities=['dFoF'], protocol_id=0, verbose=False)
    
        resp = compute_tuning_response_per_cells(data, Episodes,
                                                 contrast = contrast,
                                                 stat_test_props=stat_test,
                                                 response_significance_threshold=response_significance_threshold)
    
        DATASET['fraction_responsive'+'-c=%.1f'%contrast].append(np.sum(resp['significant_ROIs'])/len(resp['significant_ROIs']))
        DATASET['responsive'+'-c=%.1f'%contrast].append(resp['significant_ROIs'])
        DATASET['mean-selectivity'+'-c=%.1f'%contrast].append(np.mean(resp['selectivities']))
        DATASET['sd-selectivity'+'-c=%.1f'%contrast].append(np.std(resp['selectivities']))
        DATASET['tuning-curves'+'-c=%.1f'%contrast].append(np.clip(resp['Responses'], 0, np.inf))
    
DATASET['shifted_angle'] = resp['shifted_angle']

In [ ]:
keys = ['fraction_responsive', 'mean-selectivity', 'sd-selectivity']
X = range(16, int(np.max(DATASET['ages']))+1)
    
for contrast in [0.5, 1.0]:
    
    fig, AX = pt.figure(axes=(len(keys),1), figsize=(1.2,1.1), top=1.5)
    fig.suptitle('contrast = %.1f ' % contrast)
    
    for ax, key, title  in zip(AX, keys, ['responsiveness', 'mean selectivity', 's.d. selectivity']):
        ax.set_xticks(X)
        pt.scatter(DATASET['ages'], 100.*np.array(DATASET[key+'-c=%.1f'%contrast]), ax=ax, ms=5)
        pt.set_plot(ax, xticks=X[::2], xminor_ticks=X, yticks=[0,50,100], ylabel=' (%)', title=title,
                    xticks_labels=['P%i'%x for x in X[::2]], xticks_rotation=50)

In [ ]:
ages = np.unique(DATASET['ages'])

Intervals = [[16,19],
             [20,23],
             [24,27]]


DATASET['shifted_angle'] = resp['shifted_angle']

for contrast in [0.5, 1.0]:
    
    fig, AX = pt.figure(axes=(len(Intervals), 2), wspace=2., hspace=0.6, top=2.)
    fig.suptitle('contrast = %.1f ' % contrast)


    for i, age in enumerate(Intervals):
    
        cond = (DATASET['ages']>=age[0]) & (DATASET['ages']<=age[1])
    
        # responsiveness
        responsive = np.concatenate([r for r, c in zip(DATASET['responsive'+'-c=%.1f'%contrast], cond) if c])
        r = np.sum(responsive)/len(responsive)
    
        inset = pt.inset(AX[0][i], [0.8,0.8,0.6,0.6])
        pt.pie([r, 1-r], COLORS=['tab:green', 'tab:grey'], ax=inset)
        inset.set_title('%.1f%%' %(100*r), color='tab:green')
        pt.annotate(AX[0][i], '%i ROIs\n' % len(responsive), (0, 1), ha='right')
    
        # tuning raw
        tuning = np.concatenate([r for r, c in zip(DATASET['tuning-curves'+'-c=%.1f'%contrast], cond) if c])
        pt.plot(DATASET['shifted_angle'], np.mean(tuning, axis=0), sy=np.std(tuning, axis=0), ax=AX[0][i])
    
        # tuning normalized
        tuning = np.concatenate([r for r, c in zip(DATASET['tuning-curves'+'-c=%.1f'%contrast], cond) if c])
        tuning = np.transpose(np.array(tuning).T/np.max(tuning, axis=1))
        pt.plot(DATASET['shifted_angle'], np.mean(tuning, axis=0), sy=np.std(tuning, axis=0), ax=AX[1][i])
        
        pt.set_plot(AX[0][i], title='P%i-P%i' % (age[0], age[1]), ylabel='$\\Delta$F/F', 
                    xticks=DATASET['shifted_angle'], xticks_labels=[])
        pt.set_plot(AX[1][i], ylabel='norm. $\\Delta$F/F', xlabel='angle from pref. ($^{o}$)',
                    xticks=DATASET['shifted_angle'],  yticks=[0, 0.5, 1], ylim=[0,1],
                    xticks_labels=['%.0f'%s if (i%4==1) else '' for i,s in enumerate(DATASET['shifted_angle'])])

In [ ]:
ages = np.unique(DATASET['ages'])


from scipy.stats import sem
Intervals = [[16,19],
             [20,23],
             [24,27]]

from scipy.optimize import minimize

def func(S, X):
    """ fitting function """
    nS = (S+90)%180-90
    return X[0]*np.exp(-(nS**2/2./X[1]**2))+X[2]
x = np.linspace(-30, 180-30, 100)

fig, AX = pt.figure(axes=(len(Intervals), 1), wspace=2., hspace=0.6, top=2.)
for i, age in enumerate(Intervals):
    for contrast, color in zip([0.5, 1.0], ['tab:grey', 'k']):
    
        cond = (DATASET['ages']>=age[0]) & (DATASET['ages']<=age[1])
        # tuning normalized
        tuning = np.concatenate([r for r, c in zip(DATASET['tuning-curves'+'-c=%.1f'%contrast], cond) if c])
        tuning = np.transpose(np.array(tuning).T/np.max(tuning, axis=1))
        def to_minimize(x0):
            return np.sum((tuning.mean(axis=0)-\
                           func(DATASET['shifted_angle'], x0))**2)
        res = minimize(to_minimize, [0.8, 10, 0.2])
        AX[i].plot(x, func(x, res.x), lw=2, alpha=.5, color=color)

        pt.scatter(DATASET['shifted_angle'], np.mean(tuning, axis=0), sy=sem(tuning, axis=0), color=color, ax=AX[i], ms=2)
        pt.annotate(AX[i], ' c=%.1f'%contrast, (.8,1), va='top', ha='right' if color!='k' else 'left', color=color, fontsize=7)
        
    pt.set_plot(AX[i], ylabel='norm. $\\Delta$F/F', xlabel='angle from pref. ($^{o}$)',
                title='P%i-P%i\n' % (age[0], age[1]), 
                xticks=DATASET['shifted_angle'],  yticks=[0, 0.5, 1], ylim=[-0.1,1.1],
                xticks_labels=['%.0f'%s if (i%4==1) else '' for i,s in enumerate(DATASET['shifted_angle'])])